In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt


In [3]:
model_path = "model/llama-2-7b-chat.Q4_0.gguf"
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_path=model_path,
    temperature=0,
    max_new_tokens=2048,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 0},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from model/llama-2-7b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_0     [  4096,  4096,     1,    

# Query engine set up with LlamaCPP


In [4]:
# use a different LLM
embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/Users/lkp/miniforge3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# create a service context
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    chunk_overlap=30,
)


In [6]:
documents = SimpleDirectoryReader("data").load_data()


In [7]:
# create vector store index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)


In [8]:
index.storage_context.persist()

In [9]:
# define the query engine from the vector index
query_engine = index.as_query_engine()


In [11]:
response = query_engine.query("How do I load a shapefile in QGIS?")
print(response)

Llama.generate: prefix-match hit


  To load a shapefile in QGIS, you can follow these steps:
1. Open the Data Source Manager dialog by clicking on the "OpenDataSourceManager" button or by selecting it from the "Layer" menu.
2. In the Data Source Manager dialog, enable the "Vector" tab and click the "Browse" button to select the shapefile you want to load.
3. Select the shapefile and press "Add". The layer will be added using the native mesh rendering.
4. If the selected file contains many mesh layers, you may be prompted with a dialog to choose the sublayers to load. Do your selection and press OK.
5. You can also import a limited text file (e.g. .txt, .csv, .dat, .wkt) using the "Add Delimited Text Layer" tool. Click the "OpenDataSourceManager" icon to open the Data Source Manager dialog, enable the "Delimited Text" tab, and select the text file you want to import by clicking the "Browse" button. In the "Layer name" field, provide a name for the layer in your project (e.g. "Elevation"). Configure the settings as neede


llama_print_timings:        load time =  5095.64 ms
llama_print_timings:      sample time =   424.30 ms /   281 runs   (    1.51 ms per token,   662.27 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 22881.52 ms /   281 runs   (   81.43 ms per token,    12.28 tokens per second)
llama_print_timings:       total time = 24040.72 ms


In [15]:
r_query_engine = index.as_query_engine(streaming=True)
response_iter = r_query_engine.query("How do I load TIFF files in QGIS 3.22?")
response_iter.print_response_stream()

Llama.generate: prefix-match hit


  To load TIFF files in QGIS 3.22, you can follow these steps:
1. Open the Data Source Manager dialog by clicking on "Open Data Source Manager" or pressing Ctrl+L.
2. In the layer type tab, select "Raster" from the drop-down menu.
3. Click on the "Add Raster Layer" button or menu option, or use the keyboard shortcut Ctrl+Shift+R.
4. Select the TIFF file you want to load by navigating through your file system and clicking on the desired file. You can hold down the Ctrl key to select multiple files at once, or Shift key to select a range of files.
5. Once you have selected the TIFF file, click "Open" to load it into QGIS. The file will be added as a new raster layer in the Data Source Manager dialog.
Note: If you are trying to load a TIFF file that is not well-formatted or has errors, you may need to use the "Fix Layer" tool in the "Raster" menu to fix any problems before loading it into QGIS.


llama_print_timings:        load time =  8872.57 ms
llama_print_timings:      sample time =   574.16 ms /   253 runs   (    2.27 ms per token,   440.64 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 19626.61 ms /   253 runs   (   77.58 ms per token,    12.89 tokens per second)
llama_print_timings:       total time = 21661.03 ms


In [47]:
response_iter = r_query_engine.query("Using PyQGIS only, create a script that will load a TIFF file and clip it to the bounds of shapefile. Don't include the explanations.")
response_iter.print_response_stream()

Llama.generate: prefix-match hit


  Here is a script that can be used with PyQGIS to load a TIFF file and clip it to the bounds of a shapefile:
```
from pyqgis.processing import Processing
# Load the TIFF file
tiff_file = "path/to/TIFF/file.tif"

# Create a new QGIS project
project = qgis.Project()

# Load the shapefile
shapefile = "path/to/shapefile.shp"

# Define the clipping extent
extent = qgis.Extent(xmin=0, ymin=0, xmax=100, ymax=100)

# Clip the TIFF file to the shapefile
output = process.clipvectorbyextent(tiff_file, ent)


# Save the clipped TIFF file
output.save("path/to/clipped/TIFF/file.tif")
```


llama_print_timings:        load time =  4950.71 ms
llama_print_timings:      sample time =   510.26 ms /   223 runs   (    2.29 ms per token,   437.03 tokens per second)
llama_print_timings: prompt eval time =  8025.03 ms /   844 tokens (    9.51 ms per token,   105.17 tokens per second)
llama_print_timings:        eval time = 17540.61 ms /   222 runs   (   79.01 ms per token,    12.66 tokens per second)
llama_print_timings:       total time = 27393.06 ms
